In [1]:
# Artificial Neural Network Churn Rate model building
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
os.chdir(r'F:\Machine learning\Machine learning practice\Minimize Churn rate')

In [2]:
dataset = pd.read_csv('new_churn_data.csv')
print(dataset.describe())
print(dataset.columns)

               user         churn           age      deposits    withdrawal  \
count  26996.000000  26996.000000  26996.000000  26996.000000  26996.000000   
mean   35418.535302      0.413913     32.219921      3.342051      0.307045   
std    20319.620354      0.492542      9.964838      9.131992      1.055488   
min        1.000000      0.000000     17.000000      0.000000      0.000000   
25%    17808.750000      0.000000     25.000000      0.000000      0.000000   
50%    35745.500000      0.000000     30.000000      0.000000      0.000000   
75%    53236.750000      1.000000     37.000000      1.000000      0.000000   
max    69658.000000      1.000000     91.000000     65.000000     29.000000   

       purchases_partners     purchases      cc_taken  cc_recommended  \
count        26996.000000  26996.000000  26996.000000    26996.000000   
mean            28.066677      3.273967      0.073789       92.639502   
std             42.221432      8.953651      0.437331       88.868773

In [3]:
user_id = dataset['user']
dataset = dataset.drop(columns = ['user'])

In [4]:
# One-Hot Encoding: This will introduce new columns for all the categorical columns. Very smart!!
dataset = pd.get_dummies(dataset)
dataset.columns

Index(['churn', 'age', 'deposits', 'withdrawal', 'purchases_partners',
       'purchases', 'cc_taken', 'cc_recommended', 'cc_disliked', 'cc_liked',
       'cc_application_begin', 'app_downloaded', 'web_user', 'android_user',
       'registered_phones', 'waiting_4_loan', 'cancelled_loan',
       'received_loan', 'rejected_loan', 'left_for_two_month_plus',
       'left_for_one_month', 'reward_rate', 'is_referred', 'housing_O',
       'housing_R', 'housing_na', 'payment_type_Bi-Weekly',
       'payment_type_Monthly', 'payment_type_Semi-Monthly',
       'payment_type_Weekly', 'payment_type_na', 'zodiac_sign_Aquarius',
       'zodiac_sign_Aries', 'zodiac_sign_Cancer', 'zodiac_sign_Capricorn',
       'zodiac_sign_Gemini', 'zodiac_sign_Leo', 'zodiac_sign_Libra',
       'zodiac_sign_Pisces', 'zodiac_sign_Sagittarius', 'zodiac_sign_Scorpio',
       'zodiac_sign_Taurus', 'zodiac_sign_Virgo', 'zodiac_sign_na'],
      dtype='object')

In [5]:
#prevent dummy variable trap
dataset = dataset.drop(columns = ['housing_na', 'zodiac_sign_na', 'payment_type_na'])

In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns = 'churn'), dataset['churn'],
                                                    test_size = 0.2,
                                                    random_state = 0)

In [7]:
# Balancing the Training Set
'''
suppose the training set which was prepared is having data with around 60% churn = 0
In our case we have data as such.
Churn 1 was around 12K+ and churn 0 was around 8K+. This can create a bias
Our model will be performing great even though the accuracy is 60%
But there is some bias towards churn = 0. So our DS is pretty spread out.
But in some case this doesn't happen after train test split.
Thus it is needed to perform Training set balancing
'''
print('Before balancing churn column distribution:\n' + str(y_train.value_counts()))

pos_index = y_train[y_train.values == 1].index
neg_index = y_train[y_train.values == 0].index

'''
Here we are balancing the positive and negative churns in the training set at random
'''

if len(pos_index) > len(neg_index):
    higher = pos_index
    lower = neg_index
else:
    higher = neg_index
    lower = pos_index


'''
Length of churn 0 is more than length of churn 1.
Thus we are chopping churn 0 to the length of churn 1
'''
random.seed(0)
higher = np.random.choice(higher, size=len(lower))
lower = np.asarray(lower)
new_indexes = np.concatenate((lower, higher))

X_train = X_train.loc[new_indexes,]
y_train = y_train[new_indexes]

'''
Now churn 1 count = churn 0 count = 8940 nice!!!
Run the code below
'''
print('After balancing churn column distribution:\n' + str(y_train.value_counts()))

Before balancing churn column distribution:
0    12656
1     8940
Name: churn, dtype: int64
After balancing churn column distribution:
1    8940
0    8940
Name: churn, dtype: int64


In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_X.transform(X_test))
X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values
X_train2.index = X_train.index.values
X_test2.index = X_test.index.values
X_train = X_train2
X_test = X_test2

In [11]:
X_train.shape

(17880, 40)

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 21, init = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 21, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)
classifier.save('cnn_model.h5')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=40, units=21, kernel_initializer="uniform")`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=21, kernel_initializer="uniform")`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
17880/17880 [==============================] - 8s 426us/step - loss: 0.6355 - acc: 0.6341
Epoch 2/100
17880/17880 [==============================] - 6s 358us/step - loss: 0.6161 - acc: 0.6599
Epoch 3/100
17880/17880 [==============================] - 8s 468us/step - loss: 0.6084 - acc: 0.6641
Epoch 4/100
17880/17880 [==============================] - 6s 339us/step - loss: 0.6022 - acc: 0.6702
Epoch 5/100
17880/17880 [==============================] - 6s 350us/step - loss: 0.5959 - acc: 0.6767
Epoch 6/100
17880/17880 [==============================] - 5s 303us/step - loss: 0.5917 - acc: 0.6796
Epoch 7/100
17880/17880 [==============================] - 6s 319us/step - loss: 0.5881 - acc: 0.6829
Epoch 8/100
17880/17880 [==============================] - 7s 380us/step - loss: 0.5838 - acc: 0.6846
Epoch 9/100
17880/17880 [==============================] - 5s 285us/step - loss: 0.5809 - acc: 0.6881
Epoch 10/100
17880/17880 [==============================] - 5s 275us/step - loss: 

In [23]:
from keras.models import load_model
cnn_model = load_model('cnn_model.h5')

In [31]:
y_pred = cnn_model.predict(X_test)
y_pred = (y_pred > 0.5)
y_test = (y_test == 1)
y_test

2834      True
21018    False
5590     False
15769    False
15120    False
17510    False
21136     True
7994      True
2924      True
26906    False
14574    False
15038    False
25891     True
2198     False
16766     True
17116     True
6220     False
19173    False
17473     True
16945    False
19321    False
23173    False
8046      True
25979     True
22425    False
1834      True
24661    False
7763      True
19537    False
12157     True
         ...  
1470     False
25778     True
16140    False
10589     True
11615    False
26766     True
19538     True
18317    False
20011     True
19300    False
21828     True
24591    False
6611     False
26386    False
8728     False
16706     True
3042      True
22462    False
24622    False
18678     True
2719     False
6518      True
22461    False
14486    False
22169    False
16090     True
9851     False
654       True
26248     True
10769    False
Name: churn, Length: 5400, dtype: bool

In [33]:
# Making the Confusion Matrix
# Evaluating Results
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix:\n' + str(cm))
print('accuracy_score:' + str(accuracy_score(y_test, y_pred)))
print('precision_score:' + str(precision_score(y_test, y_pred))) # tp / (tp + fp)
print('recall_score:' + str(recall_score(y_test, y_pred))) # tp / (tp + fn)
print('f1_score:' + str(f1_score(y_test, y_pred)))

Confusion matrix:
[[2044 1122]
 [ 782 1452]]
accuracy_score:0.6474074074074074
precision_score:0.5641025641025641
recall_score:0.6499552372426142
f1_score:0.6039933444259568


In [9]:
'''
Performing k fold cross validation on our deep learning model
Now k fold cross validation is in  sklearn and keras is in keras.
There is a way to combine these two which is called keras wrappers for scikitlearn.
1st: define a method which is a definition of our custom classifier
2nd: perform wrapping of keras and sklearn
3rd: perform cross_val_score for k fold cross validation.
Make sure to give argument n_jobs = -1 as DL models takes many iterations to perform k fold cross validation due to presence of epoch
This argument employs all the CPUs to make the k fold cross val
'''
import keras
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(output_dim = 21, init = 'uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dense(output_dim = 21, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
# Applying k-Fold Cross Validation
from keras.wrappers.scikit_learn import KerasClassifier as kc
from sklearn.model_selection import cross_val_score
classifier = kc(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y= y_train, cv=10, n_jobs=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=40, units=21, kernel_initializer="uniform")`
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=21, kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`


Epoch 1/100
16092/16092 [==============================] - 7s 441us/step - loss: 0.6371 - acc: 0.6276
Epoch 2/100
16092/16092 [==============================] - 5s 310us/step - loss: 0.6186 - acc: 0.6541
Epoch 3/100
16092/16092 [==============================] - 4s 266us/step - loss: 0.6096 - acc: 0.6648
Epoch 4/100
16092/16092 [==============================] - 5s 310us/step - loss: 0.6014 - acc: 0.6731
Epoch 5/100
16092/16092 [==============================] - 5s 299us/step - loss: 0.5976 - acc: 0.6754
Epoch 6/100
16092/16092 [==============================] - 5s 322us/step - loss: 0.5914 - acc: 0.6810
Epoch 7/100
16092/16092 [==============================] - 8s 488us/step - loss: 0.5865 - acc: 0.6866
Epoch 8/100
16092/16092 [==============================] - 8s 474us/step - loss: 0.5824 - acc: 0.6895
Epoch 9/100
16092/16092 [==============================] - 8s 469us/step - loss: 0.5774 - acc: 0.6946
Epoch 10/100
16092/16092 [==============================] - 8s 503us/step - loss: 

16092/16092 [==============================] - 6s 351us/step - loss: 0.5065 - acc: 0.7498
Epoch 81/100
16092/16092 [==============================] - 6s 401us/step - loss: 0.5058 - acc: 0.7470
Epoch 82/100
16092/16092 [==============================] - 8s 477us/step - loss: 0.5051 - acc: 0.7478
Epoch 83/100
16092/16092 [==============================] - 6s 347us/step - loss: 0.5054 - acc: 0.7466
Epoch 84/100
16092/16092 [==============================] - 5s 326us/step - loss: 0.5040 - acc: 0.7499
Epoch 85/100
16092/16092 [==============================] - 7s 427us/step - loss: 0.5035 - acc: 0.7496 0s - loss: 0.5031 - acc: 0
Epoch 86/100
16092/16092 [==============================] - 7s 406us/step - loss: 0.5040 - acc: 0.7491
Epoch 87/100
16092/16092 [==============================] - 4s 263us/step - loss: 0.5016 - acc: 0.7516
Epoch 88/100
16092/16092 [==============================] - 4s 278us/step - loss: 0.5036 - acc: 0.7501
Epoch 89/100
16092/16092 [==============================] -

16092/16092 [==============================] - 8s 505us/step - loss: 0.5192 - acc: 0.7389
Epoch 59/100
16092/16092 [==============================] - 8s 489us/step - loss: 0.5211 - acc: 0.7374
Epoch 60/100
16092/16092 [==============================] - 7s 463us/step - loss: 0.5176 - acc: 0.7386
Epoch 61/100
16092/16092 [==============================] - 8s 483us/step - loss: 0.5187 - acc: 0.7366
Epoch 62/100
16092/16092 [==============================] - 5s 332us/step - loss: 0.5167 - acc: 0.7404
Epoch 63/100
16092/16092 [==============================] - 4s 260us/step - loss: 0.5177 - acc: 0.7386
Epoch 64/100
16092/16092 [==============================] - 4s 238us/step - loss: 0.5159 - acc: 0.7398
Epoch 65/100
16092/16092 [==============================] - 4s 267us/step - loss: 0.5168 - acc: 0.7415
Epoch 66/100
16092/16092 [==============================] - 4s 246us/step - loss: 0.5153 - acc: 0.7419
Epoch 67/100
16092/16092 [==============================] - 4s 240us/step - loss: 0.51

16092/16092 [==============================] - 5s 317us/step - loss: 0.5260 - acc: 0.7348
Epoch 38/100
16092/16092 [==============================] - 5s 330us/step - loss: 0.5269 - acc: 0.7333
Epoch 39/100
16092/16092 [==============================] - 5s 320us/step - loss: 0.5249 - acc: 0.7343
Epoch 40/100
16092/16092 [==============================] - 5s 316us/step - loss: 0.5232 - acc: 0.7365
Epoch 41/100
16092/16092 [==============================] - 5s 296us/step - loss: 0.5225 - acc: 0.7373
Epoch 42/100
16092/16092 [==============================] - 5s 299us/step - loss: 0.5225 - acc: 0.7408
Epoch 43/100
16092/16092 [==============================] - 5s 296us/step - loss: 0.5205 - acc: 0.7404
Epoch 44/100
16092/16092 [==============================] - 5s 293us/step - loss: 0.5196 - acc: 0.7414
Epoch 45/100
16092/16092 [==============================] - 5s 293us/step - loss: 0.5204 - acc: 0.7389
Epoch 46/100
16092/16092 [==============================] - 5s 292us/step - loss: 0.51

...

16092/16092 [==============================] - 5s 306us/step - loss: 0.5454 - acc: 0.7222
Epoch 29/100
16092/16092 [==============================] - 6s 382us/step - loss: 0.5453 - acc: 0.7232
Epoch 30/100
16092/16092 [==============================] - 5s 334us/step - loss: 0.5436 - acc: 0.7252
Epoch 31/100
16092/16092 [==============================] - 5s 334us/step - loss: 0.5435 - acc: 0.7253 0s - loss: 0.54
Epoch 32/100
16092/16092 [==============================] - 6s 353us/step - loss: 0.5419 - acc: 0.7214
Epoch 33/100
16092/16092 [==============================] - 6s 354us/step - loss: 0.5414 - acc: 0.7235
Epoch 34/100
16092/16092 [==============================] - 6s 353us/step - loss: 0.5407 - acc: 0.7269
Epoch 35/100
16092/16092 [==============================] - 6s 352us/step - loss: 0.5388 - acc: 0.7278
Epoch 36/100
16092/16092 [==============================] - 6s 353us/step - loss: 0.5389 - acc: 0.7269
Epoch 37/100
16092/16092 [==============================] - 6s 353us/s

16092/16092 [==============================] - 5s 328us/step - loss: 0.5926 - acc: 0.6826
Epoch 7/100
16092/16092 [==============================] - 5s 337us/step - loss: 0.5869 - acc: 0.6899
Epoch 8/100
16092/16092 [==============================] - 5s 323us/step - loss: 0.5816 - acc: 0.6949
Epoch 9/100
16092/16092 [==============================] - 5s 337us/step - loss: 0.5769 - acc: 0.6990
Epoch 10/100
16092/16092 [==============================] - 6s 342us/step - loss: 0.5736 - acc: 0.7006
Epoch 11/100
16092/16092 [==============================] - 5s 335us/step - loss: 0.5689 - acc: 0.7071
Epoch 12/100
16092/16092 [==============================] - 6s 363us/step - loss: 0.5668 - acc: 0.7061
Epoch 13/100
16092/16092 [==============================] - 5s 314us/step - loss: 0.5641 - acc: 0.7086
Epoch 14/100
16092/16092 [==============================] - 5s 315us/step - loss: 0.5624 - acc: 0.7113
Epoch 15/100
16092/16092 [==============================] - 5s 307us/step - loss: 0.5598 

16092/16092 [==============================] - 4s 268us/step - loss: 0.5088 - acc: 0.7455
Epoch 86/100
16092/16092 [==============================] - 4s 271us/step - loss: 0.5082 - acc: 0.7447
Epoch 87/100
16092/16092 [==============================] - 4s 273us/step - loss: 0.5068 - acc: 0.7455
Epoch 88/100
16092/16092 [==============================] - 4s 267us/step - loss: 0.5076 - acc: 0.7465
Epoch 89/100
16092/16092 [==============================] - 4s 267us/step - loss: 0.5081 - acc: 0.7454
Epoch 90/100
16092/16092 [==============================] - 4s 267us/step - loss: 0.5061 - acc: 0.7464
Epoch 91/100
16092/16092 [==============================] - 4s 267us/step - loss: 0.5060 - acc: 0.7472
Epoch 92/100
16092/16092 [==============================] - 4s 268us/step - loss: 0.5062 - acc: 0.7452
Epoch 93/100
16092/16092 [==============================] - 4s 267us/step - loss: 0.5047 - acc: 0.7498
Epoch 94/100
16092/16092 [==============================] - 4s 266us/step - loss: 0.50

16092/16092 [==============================] - 4s 272us/step - loss: 0.5242 - acc: 0.7351
Epoch 64/100
16092/16092 [==============================] - 4s 270us/step - loss: 0.5240 - acc: 0.7333
Epoch 65/100
16092/16092 [==============================] - 4s 271us/step - loss: 0.5240 - acc: 0.7363
Epoch 66/100
16092/16092 [==============================] - 4s 272us/step - loss: 0.5234 - acc: 0.7369
Epoch 67/100
16092/16092 [==============================] - 4s 271us/step - loss: 0.5226 - acc: 0.7373
Epoch 68/100
16092/16092 [==============================] - 4s 271us/step - loss: 0.5224 - acc: 0.7363
Epoch 69/100
16092/16092 [==============================] - 4s 271us/step - loss: 0.5215 - acc: 0.7376
Epoch 70/100
16092/16092 [==============================] - 4s 270us/step - loss: 0.5211 - acc: 0.7385
Epoch 71/100
16092/16092 [==============================] - 4s 272us/step - loss: 0.5208 - acc: 0.7363
Epoch 72/100
16092/16092 [==============================] - 4s 272us/step - loss: 0.51

16092/16092 [==============================] - 4s 275us/step - loss: 0.5346 - acc: 0.7297
Epoch 42/100
16092/16092 [==============================] - 4s 275us/step - loss: 0.5328 - acc: 0.7333
Epoch 43/100
16092/16092 [==============================] - 4s 275us/step - loss: 0.5323 - acc: 0.7323
Epoch 44/100
16092/16092 [==============================] - 4s 277us/step - loss: 0.5310 - acc: 0.7326
Epoch 45/100
16092/16092 [==============================] - 4s 275us/step - loss: 0.5305 - acc: 0.7311
Epoch 46/100
16092/16092 [==============================] - 4s 276us/step - loss: 0.5299 - acc: 0.7330
Epoch 47/100
16092/16092 [==============================] - 4s 276us/step - loss: 0.5282 - acc: 0.7338
Epoch 48/100
16092/16092 [==============================] - 4s 275us/step - loss: 0.5284 - acc: 0.7340
Epoch 49/100
16092/16092 [==============================] - 4s 276us/step - loss: 0.5267 - acc: 0.7351 0s - loss: 0.5259 - a
Epoch 50/100
16092/16092 [==============================] - 4s 2

In [12]:
print('Checking bais variance tradeoffs')
print('accuracies mean: ' + str(accuracies.mean()))
print('accuracies std: ' + str(accuracies.std()))

Checking bais variance tradeoffs
accuracies mean: 0.615771816462545
accuracies std: 0.027076513732631396
